In [ ]:
!python -V

In [ ]:
import pandas as pd

In [ ]:
import pickle

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [ ]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("wdc-taxi-ride-fair-prediction-s3")

In [ ]:
def read_dataframe(filename):

    # Read the TXT file using '|' as the delimiter and specifying the column names
    df = pd.read_csv(filename, delimiter='|')

    # Extract the required columns
    df = df[['FAREAMOUNT', 'ORIGIN_BLOCK_LATITUDE', 'ORIGIN_BLOCK_LONGITUDE', 'DESTINATION_BLOCK_LATITUDE', 'DESTINATION_BLOCK_LONGITUDE', 'ORIGINDATETIME_TR']]

    # Convert all headers to lowercase
    df.columns = df.columns.str.lower()
    
    # Drop rows with missing values
    df = df.dropna()
    
    # Print the resulting DataFrame
    print(df.head(1))

    return df

In [ ]:
df_train = read_dataframe('../data/train/taxi_2018_01.txt')
df_val = read_dataframe('../data/val/taxi_2019_02.txt')

In [ ]:
len(df_train), len(df_val)

In [ ]:
def transform_data(df):
    
    # Convert ORIGINDATETIME_TR column to pandas datetime format
    df['origindatetime_tr'] = pd.to_datetime(df['origindatetime_tr'])

    # create new columns for month, day of week, and hour
    df['month_integer'] = df['origindatetime_tr'].dt.month
    df['dow_integer'] = df['origindatetime_tr'].dt.dayofweek
    df['hour_integer'] = df['origindatetime_tr'].dt.hour
    
   # select the final required columns
    df = df[['fareamount', 'origin_block_latitude', 'origin_block_longitude', 'destination_block_latitude', 'destination_block_longitude', 
             'month_integer', 'dow_integer', 'hour_integer']]
    
    return df

In [ ]:
df_train = transform_data(df_train)
df_val = transform_data(df_val)
categorical = ['month_integer', 'dow_integer', 'hour_integer']
numerical = ['fareamount', 'origin_block_latitude', 'origin_block_longitude', 'destination_block_latitude', 'destination_block_longitude']
dv = DictVectorizer()
train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)
target = 'fareamount'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
with mlflow.start_run():
    
    mlflow.set_tag("developer", "ahmad")
    mlflow.set_tag("model", "lin_reg")
    
    mlflow.log_param("train-data-path", "../data/train/taxi_2018_01.txt")
    mlflow.log_param("valid-data-path", "../data/val/taxi_2019_02.txt")
    
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    
    with open('models/lin_reg.bin', 'wb') as f_out:
        pickle.dump((dv, lr), f_out)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [ ]:
with mlflow.start_run():

    mlflow.set_tag("developer", "ahmad")
    mlflow.set_tag("model", "lasso")

    mlflow.log_param("train-data-path", "../data/train/taxi_2018_01.txt")
    mlflow.log_param("valid-data-path", "../data/val/taxi_2019_02.txt")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)
    
    with open('models/lasso.bin', 'wb') as f_out:
        pickle.dump((dv, lr), f_out)

    mlflow.log_artifact(local_path="models/lasso.bin", artifact_path="models_pickle")

In [ ]:
import xgboost as xgb

In [ ]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [ ]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("developer", "ahmad")
        mlflow.set_tag("model", "xgboost_hp")
        mlflow.log_param("train-data-path", "../data/train/taxi_2018_01.txt")
        mlflow.log_param("valid-data-path", "../data/val/taxi_2019_02.txt")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=2,
    trials=Trials()
)

In [ ]:
mlflow.xgboost.autolog(disable=True)

In [ ]:
with mlflow.start_run():
    mlflow.set_tag("developer", "ahmad")
    mlflow.set_tag("model", "xgboost_best")
    mlflow.log_param("train-data-path", "../data/train/taxi_2018_01.txt")
    mlflow.log_param("valid-data-path", "../data/val/taxi_2019_02.txt")
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.06004343200192765,
        'max_depth': 87,
        'min_child_weight': 0.9679753664722114,
        'objective': 'reg:linear',
        'reg_alpha': 0.006814914206560178,
        'reg_lambda': 0.004139531731897862,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=2
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")
    
    with open('models/xgboost_best.bin', 'wb') as f_out:
        pickle.dump((booster), f_out)

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")